DiffDIS
====

**High-Precision Dichotomous Image Segmentation via Probing Diffusion Capacity**

 * Paper: https://arxiv.org/abs/2410.10105

![DiffDIS Overview](../assets/diffdis_overview.png)



! Installation

```bash
git clone https://github.com/qianyu-dlut/DiffDIS.git
cd DiffDIS

pip install torch==2.2.0 torchvision==0.17.0 torchaudio==2.2.0 --index-url https://download.pytorch.org/whl/cu118
pip install -r requirements.txt
pip install -e diffusers-0.30.2/
```



 * Download pre-trained model (sd-ultra)
```python
...: from huggingface_hub import snapshot_download
...: 
...: # Replace this with your target directory
...: local_dir = "./models/sd-turbo"
...: 
...: # Download entire repository snapshot
...: snapshot_download(
...:     repo_id="stabilityai/sd-turbo",
...:     local_dir=local_dir,
...:     local_dir_use_symlinks=False  # Set to False to copy files instead of symlinks
...: )
```

 * Download the checkpoint from [Google Drive](https://drive.google.com/drive/folders/1NKmUbn9BiV7xYy_1c2khIBAuOQNuYAdR)

```bash
pip install gdown

# Right click on the file and get the link

gdown https://drive.google.com/uc?id=FILE_ID
```

In [2]:
import os
import sys

import numpy as np
import torch
from PIL import Image
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

from torchvision import transforms
from transformers import CLIPTextModel, CLIPTokenizer

sys.path.append("DiffDIS-repo")
from core.diffdis_pipeline import DiffDISPipeline
from diffusers import (
    DDPMScheduler,
    UNet2DConditionModel_diffdis,
    AutoencoderKL,
)

from utils.seed_all import seed_all 
from utils.utils import check_mkdir
from utils.config import diste1,diste2,diste3,diste4,disvd
from utils.image_util import resize_res

to_pil = transforms.ToPILImage()
